In [211]:
from transformers import BertModel, BertForMaskedLM, BertConfig


small_bert_configuration = BertConfig(
    vocab_size=100, 
    hidden_size=50, 
    num_hidden_layers=2, 
    num_attention_heads=2,
    intermediate_size=200,
    max_position_embeddings=200,
    return_dict=True,
    
)

model = BertModel(small_bert_configuration)


model = BertForMaskedLM(small_bert_configuration)
print("done")

done


In [175]:
model.train()
print("putting in training mode")

putting in training mode


In [176]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

In [194]:
from ipywidgets import IntProgress
from transformers import BertTokenizer
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


tokenizer = BertTokenizer(vocab_file="/Users/irbraun/Desktop/v.txt", pad_token="[PAD]")



print(tokenizer)
print(tokenizer.pad_token_id)





text_batch = [
    "stop this madness", 
    "you do not care",
    "something about this is not right at all",
    "something is not right about this"
]
encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


PreTrainedTokenizer(name_or_path='', vocab_size=16, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
0


In [ ]:
tokenizer()

In [178]:
encoding.input_ids

tensor([[ 2,  4,  5,  6,  1,  0,  0,  0,  0,  0],
        [ 2,  7,  8,  9, 10,  1,  0,  0,  0,  0],
        [ 2, 11, 12,  5, 13,  9, 14,  2, 15,  1],
        [ 2, 11, 13,  9, 14, 12,  5,  1,  0,  0]])

In [179]:
encoding.token_type_ids

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [180]:
encoding.attention_mask

tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])

In [142]:
from torch.utils.data import TensorDataset
dataset = TensorDataset(encoding.input_ids, encoding.attention_mask)
dataset

In [133]:
from torch.utils.data import DataLoader, RandomSampler
batch_size = 4
train_dataloader = DataLoader(dataset, sampler=RandomSampler(dataset), batch_size=batch_size)

In [134]:
model.num_parameters()

79250

In [181]:
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 42 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                     (100, 50)
bert.embeddings.position_embeddings.weight                 (200, 50)
bert.embeddings.token_type_embeddings.weight                 (2, 50)
bert.embeddings.LayerNorm.weight                               (50,)
bert.embeddings.LayerNorm.bias                                 (50,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight            (50, 50)
bert.encoder.layer.0.attention.self.query.bias                 (50,)
bert.encoder.layer.0.attention.self.key.weight              (50, 50)
bert.encoder.layer.0.attention.self.key.bias                   (50,)
bert.encoder.layer.0.attention.self.value.weight            (50, 50)
bert.encoder.layer.0.attention.self.value.bias                 (50,)
bert.encoder.layer.0.attention.output.dense.weight          (50, 50)
bert.encoder.layer.0.attention.output.dense.bias               (

In [ ]:
train_dataset = 



from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )





In [124]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [138]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 3

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).

#total_steps = len(train_dataloader) * epochs
total_steps = 1*epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [143]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./EsperBERTo",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

trainer.train()






Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


ValueError: only one element tensors can be converted to Python scalars

In [216]:
inputs = tokenizer("something about this is not right at [MASK].", return_tensors="pt", padding=True, truncation=True)
labels = tokenizer("something about this is not right at all", return_tensors="pt", padding=True, truncation=True)["input_ids"]



encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']


text_batch = [
    "stop this madness", 
    "you do not care",
    "something about this is not right at all",
    "something about this is not right at [MASK]",
    "something is not right about this"]


inputs = tokenizer(text_batch[2], return_tensors="pt")
print(inputs["input_ids"].shape)
print(inputs["input_ids"])

labels = tokenizer(text_batch[3], return_tensors="pt")
print(labels["input_ids"].shape)
print(labels["input_ids"])
labels = labels["input_ids"]

model.train()
outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits
print(loss)
print(logits)

torch.Size([1, 10])
tensor([[ 2, 11, 12,  5, 13,  9, 14,  2, 15,  1]])
torch.Size([1, 10])
tensor([[ 2, 11, 12,  5, 13,  9, 14,  2,  3,  1]])
tensor(4.6353, grad_fn=<NllLossBackward>)
tensor([[[ 1.7337e-01,  5.6298e-02, -1.2305e-02, -1.5025e-02,  1.6851e-01,
          -7.2564e-02,  3.3484e-01, -2.1791e-01,  8.8291e-02, -1.0965e-01,
           4.6384e-02,  3.9461e-02,  9.9061e-03, -4.3853e-02, -2.2948e-01,
          -3.4744e-01, -2.7318e-01, -2.3657e-02,  4.2941e-01, -3.2415e-02,
          -1.4915e-02, -2.1313e-02, -2.4112e-02,  1.7946e-01, -4.2223e-02,
          -3.2972e-02,  3.0619e-02, -2.4226e-01, -1.8104e-01, -2.4036e-02,
           2.0741e-01,  9.9586e-02, -1.7982e-01,  6.9737e-03, -1.4867e-01,
          -1.2062e-01, -1.5489e-01, -2.4821e-03,  3.4308e-02, -1.7115e-01,
          -1.2163e-01, -9.3340e-02, -7.3554e-03,  2.0202e-01,  1.5587e-01,
           9.4902e-02,  7.0439e-02,  1.1978e-02,  1.2689e-01, -3.6209e-02,
          -3.0630e-01,  1.3450e-01,  1.1919e-01, -2.4746e-01,  1.4

In [245]:
from torch.utils.data import TensorDataset, random_split


text_batch = [
    "stop this madness", 
    "you do not care",
    "something about this is not right at all",
    "something is not right about this"]

text_batch_masked = [
    "stop this [MASK]", 
    "you do not [MASK]",
    "something about this is not right at [MASK]",
    "something is not right about [MASK]"]




#inputs = tokenizer(text_batch, return_tensors="pt")
labels_dict = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True)
inputs_dict = tokenizer(text_batch_masked, return_tensors='pt', padding=True, truncation=True)

print(inputs_dict)

print(inputs_dict["input_ids"].shape)
print(labels_dict["input_ids"].shape)

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(inputs_dict["input_ids"], inputs_dict["attention_mask"], labels_dict["input_ids"])


from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 32
train_dataloader = DataLoader(
    dataset,  # The training samples.
    sampler = RandomSampler(dataset), # Select batches randomly
    batch_size = batch_size # Trains with this batch size.
)

{'input_ids': tensor([[ 2,  4,  5,  3,  1,  0,  0,  0,  0,  0],
        [ 2,  7,  8,  9,  3,  1,  0,  0,  0,  0],
        [ 2, 11, 12,  5, 13,  9, 14,  2,  3,  1],
        [ 2, 11, 13,  9, 14, 12,  3,  1,  0,  0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])}
torch.Size([4, 10])
torch.Size([4, 10])


In [262]:
# The training loop that uses the data loader constructed above.
epochs = 20
for epoch_i in range(0, epochs):
    model.train()
    total_train_loss =0 
    for step,batch in enumerate(train_dataloader):
        

        model.zero_grad()
        outputs = model(input_ids=batch[0], attention_mask=batch[1], labels=batch[2])
        loss = outputs.loss
        logits = outputs.logits
        #print(loss)
        #print(logits)
        
        
        print(loss)
        
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        scheduler.step()
        optimizer.step()
        
    avg_train_loss = total_train_loss / len(train_dataloader) 
    print(avg_train_loss)


    
    
    

tensor(4.6035, grad_fn=<NllLossBackward>)
4.603512763977051
tensor(4.5893, grad_fn=<NllLossBackward>)
4.589305400848389
tensor(4.5936, grad_fn=<NllLossBackward>)
4.593636512756348
tensor(4.6005, grad_fn=<NllLossBackward>)
4.600455284118652
tensor(4.5967, grad_fn=<NllLossBackward>)
4.596670150756836
tensor(4.6087, grad_fn=<NllLossBackward>)
4.608651161193848
tensor(4.5965, grad_fn=<NllLossBackward>)
4.596545219421387
tensor(4.5860, grad_fn=<NllLossBackward>)
4.5860395431518555
tensor(4.5896, grad_fn=<NllLossBackward>)
4.589615345001221
tensor(4.5974, grad_fn=<NllLossBackward>)
4.597408294677734
tensor(4.5844, grad_fn=<NllLossBackward>)
4.584423065185547
tensor(4.6066, grad_fn=<NllLossBackward>)
4.606635093688965
tensor(4.5944, grad_fn=<NllLossBackward>)
4.594357967376709
tensor(4.6038, grad_fn=<NllLossBackward>)
4.603768348693848
tensor(4.5964, grad_fn=<NllLossBackward>)
4.596415996551514
tensor(4.5886, grad_fn=<NllLossBackward>)
4.588630676269531
tensor(4.5823, grad_fn=<NllLossBackward

In [ ]:
import time
import datetime
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [104]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/Users/irbraun/Desktop/s.txt",
    block_size=128,
)

In [105]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

data_collator

DataCollatorForLanguageModeling(tokenizer=PreTrainedTokenizer(name_or_path='', vocab_size=17, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), mlm=True, mlm_probability=0.15)

In [106]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./EsperBERTo",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [107]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


TypeError: forward() got an unexpected keyword argument 'labels'

In [56]:
labels = torch.tensor([1,0]).unsqueeze(0)
outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
loss = outputs.loss
loss.backward()
optimizer.step()

TypeError: forward() got an unexpected keyword argument 'labels'

In [54]:
from transformers import get_linear_schedule_with_warmup
num_warmup_steps = 5
num_train_steps = 10
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_train_steps)

In [55]:
loss.backward()
optimizer.step()
scheduler.step()

NameError: name 'loss' is not defined